$\textbf{What the simulation is}$

- It is a simulation of Brownian motion, or the stochastic motion of fluid particles.
- It is made in the Julia framework, which offers a lot of open-source libraries while still offering computational tools comparable to MATLAB.
- Generates an environment with small water molecules interacting with a larger tracking molecule.
- Prescribes random motion to the environment.
- Also applies a restorative force to the tracking particle.
    
$\textbf{How it works}$

- Initial GUI provides user control over simulation parameters.
- Each particle is its own "object" with its own fields.
- Defines area of red cylinder, $A_c$, and removes this from the simulation domain, $\Omega$. Iterative algorithm then defines proper mesh grid for small particle generation on the new domain $\Omega - A_c$
- Particle collision detection uses origin as a constant reference point for each iteration.
- Small time-step --> high accuracy numerical approximations and interpolations.
- Sorting algorithms to (theoretically) increase time efficiency.
- System dynamics evolves iteratively by implementing governing equations based on current behavior.
    
$\textbf{Current issues}$

- The sorting algorithms used were placeholders at the time to check functionality, but have now become outdated and inefficient.
- Collision detection is not always applied for the full length of a single time step, which can cause potentially catastrophic computational errors.
- There are some bugs that will inherently occur when the user inputs unrealistic parameters. There should be a system that prevents such simulations from running.


In [1]:
using Interact, LaTeXStrings
firstOption = 0; secondOption = 0; thirdOption = 0; fourthOption = 0; fifthOption = 0; sixthOption = 0;
function updateSim()
    global firstOption = textbox("", label="Number of particles:");
    global secondOption = dropdown(["Solid Boundary Condition", "Periodic Boundary Condition"]);
    global thirdOption = textbox("", label="Wall Length (Angstrom)");
    global fourthOption = textbox("", label="Temperature (K):");
    global fifthOption = textbox("", label="Spring constant (mN/m):");
    global sixthOption = textbox("", label="Cylinder Radius (Angstrom)");
    update = button("Update")
    output = Interact.@map (&update)
    wdg = Widget(["one" => firstOption, "two" => secondOption, "three" => thirdOption, "four" => fourthOption, "five" => fifthOption, "six" => sixthOption, "update" => update], output = output)
    @layout! wdg hbox(vbox(:one, :two, :three, :four, :five, :six, :update))
end
updateSim()

WebIO._IJuliaInit()

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["Number of particles:"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:attributes => Dict{Any, Any}(:type => "text", Symbol("data-bind") => "value: value, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}"), :placeholder => "", :className => "input ", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{Observables.AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing), "value" => (Observable{String} with 1 listeners. Value:
"", nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\ctayl\\.julia\\packages\\Knockout\\3gjS1\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\ctayl\\.julia\\packages\\Knockout\\3gjS1\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\ctayl\\.julia\\packages\\InteractBase\\h8Fwe\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\ctayl\\.julia\\packages\\InteractBase\\h8Fwe\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\ctayl\\.julia\\packages\\Interact\\pVyN2\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "value" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"value\"]()) ? (this.valueFromJulia[\"value\"]=true, this.model[\"value\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000000cc243f0, Task (runnable) @0x000000000cc243f0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"8086995066650528032\",\"id\":\"5961013806240689302\",\"type\":\"observable\"}),\"value\":WebIO.getval({\"name\":\"value\",\"scope\":\"8086995066650528032\",\"id\":\"17677697976155306061\",\"type\":\"observable\"})};\n    var self = this

In [3]:
using Plots, LinearAlgebra, Random

k_B = 1.380649E-23;       # Boltzmann constant
N_A = 6.02214076E23;      # Avogadro's number

# Variable declarations
particles = 0;            # particle array
particleRadius = 2.75;         # Radius of our particles, units are based on the "ticks" of our grid (which are not shown)
cylinderRadius = parse(Float64, sixthOption[]); # Radius of the particle in our Lagrangian frame of reference
lagrangePath = [];        # Holds all previous locations of the particle we're following to plot its path
distFromStart = [];       # Keeps track of the distance of the Lagrange particle from its starting position
cylinderMapLocation = 1;

myEnv = 0;                # environment object
particleCount = parse(Int, firstOption[]);
frameCount = 200;
boundaryCondition = secondOption[];
boxSize = parse(Float64, thirdOption[]);            # This refers to how many ticks our box has
effectiveTemp = parse(Float64, fourthOption[]);      # The effective temperature of the environment
numPixels = 600;          # This scales how large we want our box to actually be
timeVec = [0];
springConst = 10^(-3)*parse(Float64, fifthOption[]);

distanceMap = 0;          # Keeps track of the distance of every particle in relation to our lagrangian particle
framesPerSec = 60;        # Higher fps = better, less buggy simulation

uncertaintyParam = 0.5;  # The lower this parameter, the less "buggy" the simulation will behave, with the tradeoff
                          # being that the simulation takes longer to load. This method of quality control will be used
                          # until a better method of collision detection is implemented


# Structures
mutable struct particle{Float64} # Particle structure, contains position, velocity, and size
    x::Float64
    y::Float64
    v_x::Float64
    v_y::Float64
    size::Float64
    mass::Float64
end

mutable struct environment # Environment structure, contains size of environment and number of particles within it
    numParticles::Int64
    length
    height
    numSteps::Int64
    effTemp
end

# Functions
# this function initializes the environment
function createEnv(nP, l, h, nS, eT)
    global myEnv = environment(nP, l, h, nS, eT);
end

# this function spawns an appropriate number of particles
# into the environment. Each particle has a random position and
# velocity, but maintains a set size.
function populateEnv()
    global particles = Array{particle}(undef, myEnv.numParticles, 1);
    global distanceMap = Array{Float64}(undef, myEnv.numParticles, 1);
    global timeVec = [0];
    
    # Spawn the cylinder in the center of the box
    particles[1] = particle(0., 0., 0., 0., 0., 0.);
    particles[1].x = myEnv.length/2;
    particles[1].y = myEnv.length/2;
    global distFromStart = [0];
    global lagrangePath = [particles[1].x; particles[1].y];
    particles[1].v_x = 0;
    particles[1].v_y = 0;
    particles[1].size = cylinderRadius;
    particles[1].mass = 1000/N_A;
    
    # The following algorithm is used to generate a discretized domain that excludes the area occupied by
    # the cylinder:
    #  - We know the minimal mesh size needed to spawn every particle in the grid
    #     - Thus, we want the further refine the mesh so that the mesh size is also small enough to have enough
    #       mesh squares that also don't overlap the cylinder
    #  - Iteratively generate finer mesh size until the number of free mesh squares > number of particles
    #  - Compute floor((#mesh squares)/(# particles))
    #     - We will skip over this many squares on our initial pass through of spawning particles to ensure they
    #       are uniformly distributed throughout the environment
    #  - After the initial pass, randomly spawn remaining particles into remaining free mesh squares
    
    meshCompleted = false;
    numSquaresAdded = 0;
    skipSpaces = 0;
    boxLength = 0;
    numSquares = 0;
    numSquaresInRow = 0;
    while(!meshCompleted)
        numSquares = (ceil(sqrt(myEnv.numParticles))+numSquaresAdded)^2;
        numSquaresInRow = sqrt(numSquares)
        boxLength = myEnv.length/numSquaresInRow;
        overlapSquares = 0;
        # We use division and modulo to iterate through the mesh with only one variable
        for i ∈ 0:numSquares-1
            # If the distance between the center of the box and the center of the particle is less than the
            # maximum distance the two could be and still have overlap
            rowNum = floor(i/numSquaresInRow); box_x = (rowNum+1/2)*boxLength; c_x = particles[1].x;
            colNum = i%numSquaresInRow; box_y = (colNum+1/2)*boxLength; c_y = particles[1].y;
            distFromCylinder = sqrt((box_x - c_x)^2 + (box_y - c_y)^2);
            fundamentalAngle = angle(abs((box_y - c_y)), abs((box_x - c_x)));
            closestAngle = abs(pi/2*round(fundamentalAngle/(pi/2)) - fundamentalAngle);
            if( distFromCylinder <= particles[1].size + boxLength*sqrt(2)*closestAngle )
                # Add to the unusuable counter
                overlapSquares = overlapSquares + 1;
                
            end
        end
        if( numSquares - overlapSquares < myEnv.numParticles )
            numSquaresAdded = numSquaresAdded + 1;
        else
            meshCompleted = true;
            skipSpaces = floor(numSquares/myEnv.numParticles) + 1;
        end
    end

    particleNum = 1;
    usedPositions = [];
    for i in 1:skipSpaces:numSquares-1
        # Create a new particle with random properties under the constraints that it must be within the current box
        # and it has a pre-determined size
        p = particle(0., 0., 0., 0., 0., 0.);
        rowNum = floor(i/numSquaresInRow); box_x = (rowNum+1/2)*boxLength; c_x = particles[1].x;
        colNum = i%numSquaresInRow; box_y = (colNum+1/2)*boxLength; c_y = particles[1].y;
        distFromCylinder = sqrt((box_x - c_x)^2 + (box_y - c_y)^2);
        fundamentalAngle = angle(abs((box_y - c_y)), abs((box_x - c_x)));
        closestAngle = abs(pi/2*round(fundamentalAngle/(pi/2)) - fundamentalAngle);
        # If there is no overlap between the current box and the cylinder, then we can spawn a particle in this box
        if( distFromCylinder > particles[1].size + boxLength*sqrt(2)*closestAngle )
            
            particleNum = particleNum + 1;
            
            if(i == 1)
                usedPositions = [i];
            else
                usedPositions = hcat(usedPositions, [i]);
            end
            
            p.x = round( (rowNum * boxLength + particleRadius) + (boxLength - 2*particleRadius)*rand(), digits = 5 );
            p.y = round( (colNum * boxLength + particleRadius) + (boxLength - 2*particleRadius)*rand(), digits = 5 );
            p.size = particleRadius;
            p.mass = 18.01/N_A; # Mass of a water molecule
            meanVel = sqrt(8*(k_B*1000)*myEnv.effTemp/(pi*p.mass))*10^(-1);
            p.v_x = round( (-1 + 2*rand()) * meanVel, digits = 5 );
            p.v_y = round( (-1)^round(rand())*sqrt(meanVel^2 - p.v_x^2), digits = 5 );
            particles[particleNum] = p;
        end
    end
    
    i = 2 + round((numSquares-3)*rand());
    while(particleNum <= myEnv.numParticles)
        p = particle(0., 0., 0., 0., 0., 0.);
        rowNum = floor(i/numSquaresInRow); box_x = (rowNum+1/2)*boxLength; c_x = particles[1].x;
        colNum = i%numSquaresInRow; box_y = (colNum+1/2)*boxLength; c_y = particles[1].y;
        distFromCylinder = sqrt((box_x - c_x)^2 + (box_y - c_y)^2);
        fundamentalAngle = angle(abs((box_y - c_y)), abs((box_x - c_x)));
        closestAngle = abs(pi/2*round(fundamentalAngle/(pi/2)) - fundamentalAngle);
        if( !(i in usedPositions) && distFromCylinder > particles[1].size + boxLength*sqrt(2)*closestAngle )
            p.x = round( (rowNum * boxLength + particleRadius) + (boxLength - 2*particleRadius)*rand(), digits = 5 );
            p.y = round( (colNum * boxLength + particleRadius) + (boxLength - 2*particleRadius)*rand(), digits = 5 );

            p.size = particleRadius;
            p.mass = 18.01/N_A; # Mass of a water molecule

            meanVel = sqrt(8*(k_B*1000)*myEnv.effTemp/(pi*p.mass))*10^(-1);
            p.v_x = round( (-1 + 2*rand()) * meanVel, digits = 5 );
            p.v_y = round( (-1)^round(rand())*sqrt(meanVel^2 - p.v_x^2), digits = 5 );            
            particles[particleNum] = p;
            particleNum = particleNum + 1;
            usedPositions = hcat(usedPositions, [i]);
        end
        i = 2 + round((numSquares-3)*rand());
    end
end

# This function calculates the distance between the centers of two particles using the standard distance formula
function distance(p1, p2)
    return sqrt( (p2.x - p1.x)^2 + (p2.y - p1.y)^2 );
end

function distanceFromOrigin(p1)
    return sqrt( p1.x^2 + p1.y^2 );
end

function components(mag, ang)
    return [mag*cos(ang), mag*sin(ang)];
end

# This function calculates the angle of a vector quantity, from 0 to 2pi
function angle(x, y)
    if(x == 0)
        x = 1E-10;
    end
    retVal = atan(y/x);
    if(x > 0)
        if(y > 0)
            return retVal;
        else
            return 2*pi + retVal;
        end
    else
        return pi + retVal;
    end
end

# this function detects collisions between particles
function collidedWithParticle(p1, p2)
    retVal = false;
    if(distance(p1, p2) < p1.size + p2.size)
        retVal = true;
    end
    return retVal;
end

# This function detects collisions with the walls of the environment
function collidedWithWall(p1)
    retVal = -1;
    if(boundaryCondition == "Periodic Boundary Condition")
        if(p1.x <= 0)
            p1.x = myEnv.length;
            retVal = 1; # Left wall
        elseif(p1.x >= myEnv.length)
            p1.x = 0;
            retVal = 3; # Right wall
        elseif(p1.y <= 0)
            p1.y = myEnv.length;
            retVal = 2; # Bottom wall
        elseif(p1.y >= myEnv.height)
            p1.y = 0;
            retVal = 4; # Top wall
        end
    else
        if(p1.x <= p1.size)
            p1.v_x = p1.v_x * -1;
            retVal = 1; # Left wall
        elseif(p1.x >= (myEnv.length - p1.size))
            p1.v_x = p1.v_x * -1;
            retVal = 3; # Right wall
        elseif(p1.y <= p1.size)
            p1.v_y = p1.v_y * -1;
            retVal = 2; # Bottom wall
        elseif(p1.y >= (myEnv.height - p1.size))
            p1.v_y = p1.v_y * -1;
            retVal = 4; # Top wall
        end
    end
    return retVal;
end

# This function performs linear interpolation to find point y
function interp(x1, x2, x, y1, y2)
    return y1 + (y2 - y1)/(x2 - x1)*(x - x1);
end

function iterate(frameRate)
    # One of the ways I've tried to improve efficiency is implementing a sorting algorithm to the particles so that
    # collision detection will on average take O(n) operations.
    if(myEnv.numParticles > 1)
        sorted = false;
        for i in 1:myEnv.numParticles
            distanceMap[i] = distanceFromOrigin(particles[i]);
        end
        while(!sorted)
            sorted = true;
            for i in 1:myEnv.numParticles-1
                if(distanceMap[i] > distanceMap[i+1])
                    if(cylinderMapLocation == i)
                        global cylinderMapLocation = i+1;
                    elseif(cylinderMapLocation == i+1)
                        global cylinderMapLocation = i;
                    end
                    sorted = false;
                    temp = distanceMap[i];
                    distanceMap[i] = distanceMap[i+1];
                    distanceMap[i+1] = temp;
                    temp2 = particles[i];
                    particles[i] = particles[i+1];
                    particles[i+1] = temp2;
                    
                end
            end
        end
    end
    
    # First, move all particles without checking for collisions
    springForce = 10^(-20)*springConst*last(distFromStart);     # F = kx
    springAccel = springForce/particles[cylinderMapLocation].mass; # Assuming no other forces, sum(F) = ma = kx => a = kx/m
    Δv = 1/frameRate*springAccel; # mean(v) = a*Δt
    direction = 0;
    if(! (last(distFromStart) == distFromStart[1]))
        direction = angle((lagrangePath[1, 1] - lagrangePath[1, end]), (lagrangePath[2, 1] - lagrangePath[2, end])); # find the direction of this vector (points to particle's origin)
    end
    springVel = components(Δv, direction); # decompose magnitude into x and y velocities
    particles[cylinderMapLocation].v_x += springVel[1];
    particles[cylinderMapLocation].v_y += springVel[2];
    
    for i in 1:myEnv.numParticles
        particles[i].x = particles[i].x + particles[i].v_x * 1/frameRate;
        particles[i].y = particles[i].y + particles[i].v_y * 1/frameRate;
    end
    
    # Then check for collisions and update positions accordingly. This seems to be a fairly effective way of eliminating
    # issues that arise from large and/or fast collisions
    for i in 1:myEnv.numParticles
        # First look for wall collisions
        wallCollision = collidedWithWall(particles[i]);
        if(boundaryCondition == "Solid Boundary Condition")
            x_prev = particles[i].x - particles[i].v_x * 1/frameRate;
            y_prev = particles[i].y - particles[i].v_y * 1/frameRate;
            timeOfCollision = 0; # This is the time of collision relative to the previous frame
            if(wallCollision > 0)
                # Each case is essentially the same. First determine when contact was made, then move the particle
                # back to that position. Note that we move the particle back in time at this step.
                if(wallCollision == 1)
                    timeOfCollision = interp(x_prev, particles[i].x, particles[i].size, 0, 1/frameRate);
                    particles[i].x = particles[i].size + particles[i].v_x * (1/frameRate - timeOfCollision);
                    particles[i].y = particles[i].y + particles[i].v_y * (1/frameRate - timeOfCollision);
                elseif(wallCollision == 2)
                    timeOfCollision = interp(y_prev, particles[i].y, particles[i].size, 0, 1/frameRate);
                    particles[i].x = particles[i].x + particles[i].v_x * (1/frameRate - timeOfCollision);
                    particles[i].y = particles[i].size + particles[i].v_y * (1/frameRate - timeOfCollision);
                elseif(wallCollision == 3)
                    timeOfCollision = interp(x_prev, particles[i].x, myEnv.length - particles[i].size, 0, 1/frameRate);
                    particles[i].x = (myEnv.length - particles[i].size) + particles[i].v_x * (1/frameRate - timeOfCollision);
                    particles[i].y = particles[i].y + particles[i].v_y * (1/frameRate - timeOfCollision);
                elseif(wallCollision == 4)
                    timeOfCollision = interp(y_prev, particles[i].y, myEnv.height - particles[i].size, 0, 1/frameRate);
                    particles[i].x = particles[i].x + particles[i].v_x * (1/frameRate - timeOfCollision);
                    particles[i].y = (myEnv.height - particles[i].size) + particles[i].v_y * (1/frameRate - timeOfCollision);
                end

                # Finally, update the particle's position with the correct direction for however much you went back in time
                particles[i].x = particles[i].x - particles[i].v_x * (1/frameRate - timeOfCollision);
                particles[i].y = particles[i].y - particles[i].v_y * (1/frameRate - timeOfCollision);
            end
        end
        # Next look for particle collisions. General idea is to form initial hypotheses about particles that could be
        # colliding by comparing their distances to a single particle. By using the triangle rule, we can go through our
        # sorted list and find all potential collisions. We then directly compare distances to determine which, if any,
        # are actually colliding. I do this instead of just brute forcing the problem to limit the amount of times I have
        # to calculate distance, since that uses a disproportionate amount of operations, and a brute force solution would 
        # have n^2 of them.
        j = 1;
        collisions = 0;
        while(i+j <= myEnv.numParticles && (distanceMap[i+j] - distanceMap[i] < 1/uncertaintyParam*(particles[i].size + particles[i+j].size)|| abs(distanceMap[cylinderMapLocation]-distanceMap[i]<1/uncertaintyParam*(particles[cylinderMapLocation].size+particles[i].size))))
            currentDist = distance(particles[i+j], particles[i]);
            if(currentDist < particles[i].size + particles[i+j].size)
                collisions += 1;
                # For more information on how the collision dynamics are calculated, see
                # https://en.wikipedia.org/wiki/Elastic_collision
                # The general process is the same as wall collision updates, but now we use distance to interpolate the
                # time of impact, and we have to update both the particles' values.
                xi_prev = particles[i].x - particles[i].v_x * 1/frameRate;
                yi_prev = particles[i].y - particles[i].v_y * 1/frameRate;
                xij_prev = particles[i+j].x - particles[i+j].v_x * 1/frameRate;
                yij_prev = particles[i+j].y - particles[i+j].v_y * 1/frameRate;
                
                prevDist = sqrt((xij_prev - xi_prev)^2 + (yij_prev - yi_prev)^2);
                
                timeOfCollision = interp(prevDist, currentDist, particles[i].size + particles[i+j].size, 0, 1/frameRate);
                
                particles[i].x = particles[i].x - particles[i].v_x * (1/frameRate - timeOfCollision);
                particles[i].y = particles[i].y - particles[i].v_y * (1/frameRate - timeOfCollision);
                particles[i+j].x = particles[i+j].x - particles[i+j].v_x * (1/frameRate - timeOfCollision);
                particles[i+j].y = particles[i+j].y - particles[i+j].v_y * (1/frameRate - timeOfCollision);
                
                
                theta1 = angle(particles[i].v_x, particles[i].v_y);
                theta2 = angle(particles[i+j].v_x, particles[i+j].v_y);
                Δx = particles[i].x - particles[i+j].x;
                Δy = particles[i].y - particles[i+j].y;
                contactAngle = angle(Δx, Δy);
                v1 = sqrt(particles[i].v_x^2 + particles[i].v_y^2);
                v2 = sqrt(particles[i+j].v_x^2 + particles[i+j].v_y^2);
                m1 = particles[i].mass;
                m2 = particles[i+j].mass;
                
                particles[i].v_x = (v1*cos(theta1 - contactAngle)*(m1 - m2) + 2*m2*v2*cos(theta2 - contactAngle)) *
                    cos(contactAngle)/(m1 + m2) + v1*sin(theta1 - contactAngle)*cos(contactAngle + pi/2);
                particles[i].v_y = (v1*cos(theta1 - contactAngle)*(m1 - m2) + 2*m2*v2*cos(theta2 - contactAngle)) *
                    sin(contactAngle)/(m1 + m2) + v1*sin(theta1 - contactAngle)*sin(contactAngle + pi/2);
                particles[i+j].v_x = (v2*cos(theta2 - contactAngle)*(m2 - m1) + 2*m1*v1*cos(theta1 - contactAngle)) *
                    cos(contactAngle)/(m1 + m2) + v2*sin(theta2 - contactAngle)*cos(contactAngle + pi/2);
                particles[i+j].v_y = (v2*cos(theta2 - contactAngle)*(m2 - m1) + 2*m1*v1*cos(theta1 - contactAngle)) *
                    sin(contactAngle)/(m1 + m2) + v2*sin(theta2 - contactAngle)*sin(contactAngle + pi/2);
                
                particles[i].x = particles[i].x + particles[i].v_x * (1/frameRate - timeOfCollision);
                particles[i].y = particles[i].y + particles[i].v_y * (1/frameRate - timeOfCollision);
                particles[i+j].x = particles[i+j].x + particles[i+j].v_x * (1/frameRate - timeOfCollision);
                particles[i+j].y = particles[i+j].y + particles[i+j].v_y * (1/frameRate - timeOfCollision);
            end
            j = j + 1;
        end
    end
end

createEnv(particleCount, boxSize, boxSize, frameCount, effectiveTemp);
populateEnv();

#This macro does most of the heavy-lifting for us, we just pass in what we want to animate
sim = @animate for i in 1:myEnv.numSteps
    iterate(framesPerSec);
    global timeVec = hcat(timeVec, i/framesPerSec);
    global distFromStart = hcat(distFromStart, sqrt((particles[cylinderMapLocation].x - lagrangePath[1, 1])^2 + (particles[cylinderMapLocation].y - lagrangePath[2, 1])^2));
    x_vals = Array{Float64}(undef, myEnv.numParticles-1, 1);
    y_vals = Array{Float64}(undef, myEnv.numParticles-1, 1);
    sizes = Array{Float64}(undef, myEnv.numParticles-1, 1);
    for j in 1:myEnv.numParticles
        if(j < cylinderMapLocation)
            x_vals[j] = particles[j].x;
            y_vals[j] = particles[j].y;
            sizes[j] = particles[j].size;
        elseif(j > cylinderMapLocation)
            x_vals[j-1] = particles[j].x;
            y_vals[j-1] = particles[j].y;
            sizes[j-1] = particles[j].size;
        end
    end
    global lagrangePath = hcat(lagrangePath, [particles[cylinderMapLocation].x; particles[cylinderMapLocation].y]);
    plot([particles[cylinderMapLocation].x], [particles[cylinderMapLocation].y], seriestype = :scatter, size = (numPixels, numPixels), markersize = numPixels/boxSize*cylinderRadius, legend = false, xlims = [0, myEnv.length], ylims = [0, myEnv.height], grid=false, axis=([], false), color = "red");
    # plot!(lagrangePath[1, :], lagrangePath[2, :], color = "red");
    plot!(x_vals, y_vals, seriestype = :scatter, markersize = numPixels/boxSize*particleRadius, color = "blue");
    plot!([0, 0, myEnv.length, myEnv.length, 0], [0, myEnv.height, myEnv.height, 0, 0], color="black");
end;
myGif = gif(sim, "BM_Sim.mp4", fps=framesPerSec)

┌ Info: Saved animation to 
│   fn = C:\Users\ctayl\BM_Sim.mp4
└ @ Plots C:\Users\ctayl\.julia\packages\Plots\yJrrq\src\animation.jl:137


Plots.AnimatedGif("C:\\Users\\ctayl\\BM_Sim.mp4")